In [2]:
import numpy as np
import scipy.io as sio
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from scipy.signal import welch
from scipy.stats import skew, kurtosis


In [3]:
file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\featuress.mat'
feet = sio.loadmat(file_path)
Data_array = np.transpose(feet['Data_EEG'], (2, 0, 1))
Label_array = feet['Label_EEG']
Data_array.shape, Label_array.shape

((11068, 19, 300), (1, 11068))

In [3]:
def extract_mean(data):
    return np.mean(data, axis=-1)

def extract_std(data):
    return np.std(data, axis=-1)

def extract_skewness(data):
    return skew(data, axis=-1)

def extract_kurtosis(data):
    return kurtosis(data, axis=-1)

def extract_max(data):
    return np.max(data, axis=-1)

def extract_min(data):
    return np.min(data, axis=-1)

def extract_range(data):
    return np.max(data, axis=-1) - np.min(data, axis=-1)

def extract_energy(data):
    return np.sum(data**2, axis=-1)  # Energy of the signal

def extract_zero_crossing_rate(data):
    return np.sum(np.diff(np.sign(data), axis=-1) != 0, axis=-1) / (data.shape[-1] - 1)

In [4]:
feature_extractors = [
    extract_mean,
    extract_std,
    extract_skewness,
    extract_kurtosis,
    extract_max,
    extract_min,
    extract_range,
    extract_energy,
    extract_zero_crossing_rate
]

# Extract features from data array using all defined feature extractors
def extract_features(data):
    features = []
    for extractor in feature_extractors:
        extracted = extractor(data)
        if isinstance(extracted, tuple):
            features.extend(extracted)  # Unpack tuple if returned
        else:
            features.append(extracted)
    return np.concatenate(features)

# Extract features from data array
featuress = np.array([extract_features(data) for data in Data_array])

In [5]:
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics across folds
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
confusion_matrices = []

# Perform KFold cross-validation
for train_index, val_index in kf.split(featuress):
    X_train, X_val = featuress[train_index], featuress[val_index]
    y_train, y_val = Label_array[train_index], Label_array[val_index]
    
    # Apply SMOTE to balance classes in training data
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    
    # Scale features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_resampled)
    X_val_scaled = scaler.transform(X_val)
    

In [6]:
X_train_resampled.shape, y_train_resampled.shape

((12338, 171), (12338,))

In [7]:
clf = SVC(kernel='rbf', random_state=42)  # RBF kernel for SVM
clf.fit(X_train_scaled, y_train_resampled)

# Evaluate the model on the validation data
y_pred = clf.predict(X_val_scaled)

# Compute evaluation metrics
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')
f1 = f1_score(y_val, y_pred, average='weighted')
confusion = confusion_matrix(y_val, y_pred)

# Store metrics in lists
accuracy_scores.append(accuracy)
precision_scores.append(precision)
recall_scores.append(recall)
f1_scores.append(f1)
confusion_matrices.append(confusion)

# Compute mean of evaluation metrics across folds
mean_accuracy = np.mean(accuracy_scores)
mean_precision = np.mean(precision_scores)
mean_recall = np.mean(recall_scores)
mean_f1 = np.mean(f1_scores)

# Print mean evaluation metrics
print("Mean Accuracy:", mean_accuracy)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)
print("Mean F1-score:", mean_f1)

Mean Accuracy: 0.6269196025293586
Mean Precision: 0.6360276326799521
Mean Recall: 0.6269196025293586
Mean F1-score: 0.6305529925630327
